In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import threading

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
class JointDistanceManager:
    def __init__(self):
        self._curr_thumb_dis = np.array([0, 0, 0, 0])
        self._curr_index_dis = np.array([0, 0, 0, 0])
        self._curr_middle_dis = np.array([0, 0, 0, 0])
        self._curr_ring_dis = np.array([0, 0, 0, 0])
        self._curr_pinky_dis = np.array([0, 0, 0, 0])
        self.active = False

    def update(self, thumb_dis, index_dis, middle_dis, ring_dis, pinky_dis):
        self.curr_thumb_dis = np.array(thumb_dis)
        self.curr_index_dis = np.array(index_dis)
        self.curr_middle_dis = np.array(middle_dis)
        self.curr_ring_dis = np.array(ring_dis)
        self.curr_pinky_dis = np.array(pinky_dis)
        self.active = True

    def _calc_delta(self, thumb_dis, index_dis, middle_dis, ring_dis, pinky_dis):
        assert self.active == True
        thumb_dis_deltas = np.round(np.array(thumb_dis) - self.curr_thumb_dis, 2)
        index_dis_deltas = np.round(np.array(index_dis) - self.curr_index_dis, 2)
        middle_dis_deltas = np.round(np.array(middle_dis) - self.curr_middle_dis, 2)
        ring_dis_deltas = np.round(np.array(ring_dis) - self.curr_ring_dis, 2)
        pinky_dis_deltas = np.round(np.array(pinky_dis) - self.curr_pinky_dis, 2)

        return thumb_dis_deltas, index_dis_deltas, middle_dis_deltas, ring_dis_deltas, pinky_dis_deltas

    def find_rotated_joints(self, thumb_dis, index_dis, middle_dis, ring_dis, pinky_dis):
        deltas = self._calc_delta(thumb_dis, index_dis, middle_dis, ring_dis, pinky_dis)
        thumb_dis_deltas, index_dis_deltas, middle_dis_deltas, ring_dis_deltas, pinky_dis_deltas = deltas
        deltas_arr = np.array([thumb_dis_deltas, index_dis_deltas, middle_dis_deltas, ring_dis_deltas, pinky_dis_deltas])
        deltas_i, deltas_j = np.where(np.abs(deltas_arr) > 0.01)

        return deltas_i, deltas_j, deltas_arr

In [4]:
def calc_distance_of_two_adj_joints(hand_landmark, curr_joint_index):
    joint_next_to_wrist_indexes = [mp_hands.HandLandmark.THUMB_CMC.value,
                                   mp_hands.HandLandmark.INDEX_FINGER_MCP.value,
                                   mp_hands.HandLandmark.MIDDLE_FINGER_MCP.value,
                                   mp_hands.HandLandmark.RING_FINGER_MCP.value,
                                   mp_hands.HandLandmark.PINKY_MCP.value]
    if curr_joint_index in joint_next_to_wrist_indexes:
        prev_joint_index = 0
    else:
        prev_joint_index = curr_joint_index - 1

    prev_joint_index = 0
    
    current_joint = np.array([hand_landmark[curr_joint_index].x, hand_landmark[curr_joint_index].y])
    prev_joint = np.array([hand_landmark[prev_joint_index].x, hand_landmark[prev_joint_index].y])
    dist = round(np.linalg.norm(current_joint - prev_joint), 2)

    return dist

In [5]:
def hand_tracking(image):
    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
    results = hands.process(processed_image)
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))

        hand_landmark = hand.landmark
        thumb_dis, index_dis, middle_dis, ring_dis, pinky_dis = [], [], [], [], []

        """
        for i in range(1, 5):
            thumb_joint_ind = i
            index_joint_ind = thumb_joint_ind + 4
            middle_joint_ind = index_joint_ind + 4
            ring_joint_ind = middle_joint_ind + 4
            pinky_joint_ind = ring_joint_ind + 4

            thumb_dis.append(calc_distance_of_two_adj_joints(hand_landmark, thumb_joint_ind))
            index_dis.append(calc_distance_of_two_adj_joints(hand_landmark, index_joint_ind))
            middle_dis.append(calc_distance_of_two_adj_joints(hand_landmark, middle_joint_ind))
            ring_dis.append(calc_distance_of_two_adj_joints(hand_landmark, ring_joint_ind))
            pinky_dis.append(calc_distance_of_two_adj_joints(hand_landmark, pinky_joint_ind))
        """
        """
        for i, dis in enumerate(index_dis):
            if i in [0, 3]:
                continue
            coords = tuple(np.multiply(np.array((hand.landmark[i + 1 + 4].x, 
                                                 hand.landmark[i + 1 + 4].y)), 
                                        [640, 480]).astype(int))                
            cv2.putText(image, str(dis), coords, cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (255, 255, 255), 2, cv2.LINE_AA)
        """

        """
        if joint_distance_manager.active:
            rotated_fingers, rotated_joints, deltas = joint_distance_manager.find_rotated_joints(thumb_dis, 
                                                                                                 index_dis, 
                                                                                                 middle_dis, 
                                                                                                 ring_dis, 
                                                                                                 pinky_dis)
            if len(rotated_fingers):
                assert len(rotated_fingers) == len(rotated_joints)
                for finger_ind, joint_ind in zip(rotated_fingers, rotated_joints):
                    if joint_ind in [0, 3]:
                        continue
                    coords = tuple(np.multiply(np.array((hand.landmark[joint_ind + 1 + (finger_ind * 4)].x, 
                                                         hand.landmark[joint_ind + 1 + (finger_ind * 4)].y)), 
                                               [640, 480]).astype(int))                
                    cv2.putText(image, str(deltas[finger_ind, joint_ind]), coords, cv2.FONT_HERSHEY_SIMPLEX, 
                                1, (255, 255, 255), 1, cv2.LINE_AA)
                    
            
        # Update value to joint_distance_manager    
        joint_distance_manager.update(thumb_dis, index_dis, middle_dis, ring_dis, pinky_dis)
        """

    return image

In [ ]:
cap = cv2.VideoCapture(0)

#joint_distance_manager = JointDistanceManager()

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        # Flip on horizontal
        image = cv2.flip(frame, 1)

        # Set flag
        #image.flags.writeable = False

        image = hand_tracking(image.copy())
        
        cv2.imshow("Hand Tracking", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1715053428.489438  220745 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1715053428.492309  220809 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 630 (CFL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
